In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import numpy as np
import pandas as pd
import psycopg2
import psycopg2.extras
from datetime import timedelta, datetime
import time

import bb_utils
import bb_utils.meta
import bb_utils.ids
import bb_backend

bb_backend.api.server_adress = 'localhost:8000'
connect_str = """dbname='beesbook' user='reader' host='tonic.imp.fu-berlin.de' 
                 password='' application_name='mehmed'"""

In [ ]:
observ_period = timedelta(hours=1)
datetime_start = datetime(2016, 8, 23)
num_observ_periods = 24

#(First detections are on 20.07.2016, last are 19.09.2016 (3 months duration))
print('Beginning at '+datetime_start.strftime("%Y-%m-%d %H:%M:%S"))

In [ ]:
for i in range(0,num_observ_periods):
    datetime_end = datetime_start + observ_period
    datetime_str = datetime_start.strftime("%Y-%m-%d_%H:%M:%S")
    
    print('Getting hour #'+str(i)+', beginning at '+datetime_str)
    start = time.time()
    with psycopg2.connect(connect_str) as conn:
        query = """SELECT * FROM bb_detections
               WHERE timestamp >= %s AND
                     timestamp < %s AND
                     bee_id_confidence >= %s
               ;"""
    df = pd.read_sql_query(
        query, conn, 
        params=(datetime_start, datetime_end, 0.99), #note: high confidence requirement
        coerce_float=False)

    end = time.time()
    secs_elapsed = end - start

    m, s = divmod(secs_elapsed, 60)
    h, m = divmod(m, 60)
    print('Getting the data took:')
    print('%d:%02d:%02d'% (h, m, s))
    
    start = time.time()
    df.to_csv('/mnt/storage/janek/'+datetime_str+'.csv')
    datetime_start = datetime_end
    
    end = time.time()
    secs_elapsed = end - start

    m, s = divmod(secs_elapsed, 60)
    h, m = divmod(m, 60)
    print('Saving the csv took:')
    print('%d:%02d:%02d'% (h, m, s))